In [1]:
# instalar as dependências
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

In [2]:
# configurar as variáveis de ambiente
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

# tornar o pyspark "importável"
import findspark
findspark.init('spark-2.4.4-bin-hadoop2.7')

In [3]:
!pip install pyspark==3.3.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.3/281.3 MB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 15.1 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764009 sha256=e574ebaecc22493192a4af66ff04303123b5dcc78a0a85204d4ca47fe411fbbb
  Stored in directory: /root/.cache/pip/wheels/81/9c/6c/d5200fcf351ffa39cbe09911e99703283624cd037df58070d9
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [9]:
# iniciar uma sessão local e importar dados do Airbnb
from pyspark.sql import SparkSession
sc = SparkSession.builder.appName("teste").getOrCreate()
sc.conf.set('spark.sql.sources.partitionOverwriteMode', 'dynamic')

!wget --quiet --show-progress http://data.insideairbnb.com/the-netherlands/north-holland/amsterdam/2023-06-05/visualisations/listings.csv

# carregar dados do Airbnb
df = sc.read.csv("./listings.csv", inferSchema=True, header=True)


listings.csv.2      100%[===================>]   1.43M  --.-KB/s    in 0.1s    


In [22]:
from pyspark.sql.functions import *
# df2 = df.select("name")
# df2 = df2.withColumn("nome2", upper("name"))
# # df2.show(truncate=False)

# df2.createOrReplaceTempView("dataframe")
# resul = sc.sql("select * from dataframe where name like '%shared baths%'")
# resul.show(truncate=False)
# print(f'quantidade: {resul.count()}')

# df.createOrReplaceTempView('airbnb')
df2 = df.filter(df.neighbourhood == 'Centrum-West')
df2 = df2.withColumn("ano", date_format(df.last_review, "yyyy"))
df2 = df2.withColumn("mes", date_format(df.last_review, "MM"))
df2 = df2.withColumn("dia", date_format(df.last_review, "dd"))
df2.select("ano","mes", "dia").show()
df2.write.parquet('airbnb_ano_mes_dia/', partitionBy=['ano', 'mes', 'dia'], mode='overwrite')


+----+---+---+
| ano|mes|dia|
+----+---+---+
|2023| 05| 01|
|2022| 06| 26|
|2023| 06| 03|
|2022| 10| 23|
|2022| 12| 29|
|2023| 05| 19|
|2023| 06| 05|
|2022| 10| 14|
|2023| 06| 02|
|2023| 05| 17|
|2022| 12| 31|
|2023| 05| 02|
|2023| 05| 29|
|2023| 04| 29|
|2023| 05| 29|
|2023| 05| 17|
|2023| 05| 05|
|2023| 06| 01|
|2023| 05| 26|
|2022| 06| 28|
+----+---+---+
only showing top 20 rows



In [23]:
df4 = sc.read.parquet('/content/airbnb_ano_mes_dia/ano=2023')
df.printSchema()

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- host_id: integer (nullable = true)
 |-- host_name: string (nullable = true)
 |-- neighbourhood_group: string (nullable = true)
 |-- neighbourhood: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- room_type: string (nullable = true)
 |-- price: integer (nullable = true)
 |-- minimum_nights: integer (nullable = true)
 |-- number_of_reviews: integer (nullable = true)
 |-- last_review: timestamp (nullable = true)
 |-- reviews_per_month: double (nullable = true)
 |-- calculated_host_listings_count: integer (nullable = true)
 |-- availability_365: integer (nullable = true)
 |-- number_of_reviews_ltm: integer (nullable = true)
 |-- license: string (nullable = true)

